# Tools to asses the performance of our functions
Main tools I use every day:
* [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl) to benchmark the speed and memory allocations of our kernels
* Profilers to find bottlenecks (e.g. [ProfileCanvas.jl](https://github.com/pfitzseb/ProfileCanvas.jl))
* `@code_warntype` to check if our code is type stable
* [Cthulhu.jl](https://github.com/JuliaDebug/Cthulhu.jl)
* Others: [JET.jl](https://github.com/aviatesk/JET.jl), [Aqua.jl](https://github.com/JuliaTesting/Aqua.jl)

# BenchmarkTools.jl

In [1]:
using BenchmarkTools
f(x, n) = x^n
@btime f(1.2, 3.2)

  0.792 ns (0 allocations: 0 bytes)


1.7921732359737277

Be aware if you want to benchmark functions with non-literal arguments, you
need to use the dollar symbol:

In [2]:
x, n = 1.2, 3.2
@btime f($x, $n)

  11.720 ns (0 allocations: 0 bytes)


1.7921732359737277

# Type instabilities
A type instability when the compiler can't predict (infer) at compile time the types of a variable

In [3]:
function foo(x)
    v = 0.0
    for i in eachindex(x)
        v += x[i]
    end
    v
end

foo (generic function with 1 method)

If we benchmark the previous function with some funky argument, we will observe a non-expected behaviour

In [4]:
x = (1, 0.5, 34, 213.23f0)
@btime foo($x)

  56.657 ns (6 allocations: 224 bytes)


248.72999572753906

The best way to detect if and where we have type instabilities is with the `@code_warntype` macro

In [5]:
using InteractiveUtils
@code_warntype foo(x)

MethodInstance for Main.var"##1174".foo(::Tuple{Int64, Float64, Int64, Float32})
  from foo(x) @ Main.var"##1174" ~/Desktop/Seminars/JuliaMainz23/basic_perftools.ipynb:1
Arguments
  #self#::Core.Const(Main.var"##1174".foo)
  x::Tuple{Int64, Float64, Int64, Float32}
Locals
  @_3::UNION{NOTHING, TUPLE{INT64, INT64}}
  v::Float64
  i::Int64
Body::Float64
1 ─       (v = 0.0)
│   %2  = Main.var"##1174".eachindex(x)::Core.Const(Base.OneTo(4))
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3::Core.Const((1, 1)) === nothing)::Core.Const(false)
│   %5  = Base.not_int(%4)::Core.Const(true)
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Int64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = v::Float64
│   %11 = Base.getindex(x, i)::UNION{FLOAT32, FLOAT64, INT64}
│         (v = %10 + %11)
│         (@_3 = Base.iterate(%2, %9))
│   %14 = (@_3 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       r

If the argument is a homogeneous data container

In [6]:
x = (1.0, 0.5, 34.0, 213.23)

(1.0, 0.5, 34.0, 213.23)

@code_warntype foo(x)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*